# Purpose

This notebook explores the details of how pvlib determines cell temperature.  The motivation is exploring discrepencies observed in Voc and Vmpp results.

# Setup Imports

In [30]:
# Setup
import pvlib
from pvlib.pvsystem import PVSystem, Array, FixedMount
from pvlib.location import Location
from pvlib.modelchain import ModelChain
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from userdefinedmodels import *

# Setup Location and Weather

The descrepancy we observed was between a manual panel calculation and a modelchain run from 2022 weather in Phoenix.

In [31]:


# latitude, longitude, name, altitude, timezone
phoenix = (33.5, -112.0, 'Phoenix', 340, 'Etc/GMT+7')
latitude, longitude, name, altitude, timezone = phoenix

location = Location(
    latitude,
    longitude,
    name=name,
    altitude=altitude,
    tz=timezone,
    )

# read psm3 df file and setup weather
names = '2022'
filename = 'psm3_' + location.name + '_' + names + '.csv'
weather_full_year = pd.read_csv(filename, index_col=0, parse_dates=True)

# slice weather to get df with a single entry for 2022-04-08 12:10:00-07:00
weather = weather_full_year.loc['2022-04-08 12:10:00-07:00':'2022-04-08 12:10:00-07:00']

# Setup System

## Module Parameters
This evaluation will use the rich solar 12V panel

In [32]:
import richsolarpanels
rich_solar_12V_params = richsolarpanels.RICH_SOLAR_12V

## Temperature Parameters

In [33]:
temperature_model_parameters = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass']
print(temperature_model_parameters)

{'a': -3.47, 'b': -0.0594, 'deltaT': 3}


## Inverter (MPPT) Parameters

In [34]:
# We'll start with no wire losses and perfect MPPT efficiency
#r_batt_wire = getWireResistance('8AWG', 5)
#r_pv_wire = getWireResistance('8AWG', 10)
r_batt_wire = 0
r_pv_wire = 0
mppt_eff = 1.0


mppt_parameters = {
    'v_batt': 14.2,
    'v_start_delta': 5.0,
    'v_continue_delta': 1.0,
    'r_batt_wire': r_batt_wire,
    'r_pv_wire': r_pv_wire,
    'mppt_eff': mppt_eff
}

## System Configuration

In [35]:
mount = FixedMount(
    surface_tilt=0,
    surface_azimuth=180,
    )

array = Array(
    mount=mount, module_parameters=rich_solar_12V_params,
    temperature_model_parameters=temperature_model_parameters,
    strings=1, modules_per_string=1
    )

system = PVSystem(arrays=[array],inverter_parameters=mppt_parameters)

## Create the ModelChain

In [36]:
mc = ModelChain(
    system, 
    location, 
    aoi_model="physical", 
    spectral_model="no_loss", 
    dc_model="desoto",
    ac_model=mppt,
    dc_ohmic_model=pv_wire_loss
    )

# Run the Model

In [37]:
mc.run_model(weather)

ModelChain: 
  name: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: desoto
  ac_model: functools.partial(<function mppt at 0xffff59a5c9a0>, ModelChain: 
  name: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: desoto
  ac_model: ...
  aoi_model: physical_aoi_loss
  spectral_model: no_spectral_loss
  temperature_model: sapm_temp
  losses_model: no_extra_losses)
  aoi_model: physical_aoi_loss
  spectral_model: no_spectral_loss
  temperature_model: sapm_temp
  losses_model: no_extra_losses

# Print Results

In [38]:
# Print all the results
print(mc.results)
print('weather\n')
print(mc.results.weather)
print('aoi\n')
print(mc.results.aoi)
print('effective_irradiance\n')
print(mc.results.effective_irradiance)
print('total_irrad\n')
print(mc.results.total_irrad)
print('cell_temperature\n')
print(mc.results.cell_temperature)
print('diode_params\n')
print(mc.results.diode_params)
print('losses\n')
print(mc.results.losses)
print('dc\n')
print(mc.results.dc)
print('dc_ohmic_losses\n')
print(mc.results.dc_ohmic_losses)
print('ac\n')
print(mc.results.ac)

=== ModelChainResult === 
Number of Arrays: 1 
times (first 3)
DatetimeIndex(['2022-04-08 12:10:00-07:00'], dtype='datetime64[ns, UTC-07:00]', freq=None)
 ac: Series (length 1)
 airmass: DataFrame (1 rows x 2 columns)
 albedo: Series (length 1)
 aoi: Series (length 1)
 aoi_modifier: Series (length 1)
 cell_temperature: Series (length 1)
 dc: DataFrame (1 rows x 7 columns)
 dc_ohmic_losses: DataFrame (1 rows x 2 columns)
 diode_params: DataFrame (1 rows x 5 columns)
 effective_irradiance: Series (length 1)
 losses: 1
 solar_position: DataFrame (1 rows x 6 columns)
 spectral_modifier: 1
 total_irrad: DataFrame (1 rows x 5 columns)
 tracking: None
 weather: DataFrame (1 rows x 5 columns)
weather

                              ghi    dhi     dni  wind_speed  temp_air
2022-04-08 12:10:00-07:00  1016.0  107.0  1015.0         1.6      35.0
aoi

2022-04-08 12:10:00-07:00    26.485275
Name: aoi, dtype: float64
effective_irradiance

2022-04-08 12:10:00-07:00    1014.276987
dtype: float64
total_i

# ModelChain Operation



## Manual Cell Temperature Calculation

`model_chain.run_model(weather)` calls
- `self.effective_irradiance_model()` which calls
- `self._run_from_effective_irrad(weather)` which binds `weather` to `data` ___ and calls
    - `self._prepare_temperature(data)` which in our case calls
        - `self.temperature_model()` which returns
            - `self._temperature_model` which for our case was set to self.sapm_temp during init which calls
                - `self.sapm_temp()` which returns
                    - `self._set_celltemp('sapm')` which
                        - binds its model parameter to 'sapm'
                        - determines `poa = _irrad_for_cell_temp(self.results.total_irrad, self.results.effective_irradiance)` which preferentially returns `total_irrad['poa_global']` if its available else returns `effective_irradiance`
                        - determines `temp_air` and `wind_speed` from `self.results.weather`
                        - sets `self.results.cell_temperature = self.system.get_cell_temperature(poa, temp_air, wind_speed, model='sapm')` which calls
                            - `PVSystem.get_cell_temperature(poa_global, temp_air, wind_speed, model, effective_irradiance=None)` which calls for each array in `self.arrays`
                                - `array.get_cell_temperature(poa_global, temp_air, wind_speed, model, effective_irradiance)` which calls
                                    - `Array.get_cell_temperature(poa_global, temp_air, wind_speed, model, effective_irradiance=None)` which 
                                        - binds `func` to `temperature.sapm_cell`
                                        - builds required args for sapm (`a`, `b`, `deltaT`)
                                        - binds `temperature_cell` to the result of `func` called with `poa_global`, `temp_air`, `wind_speed` and the required sapm args
                                            - which calls `temperature.sapm_cell(poa_global, temp_air, wind_speed, a, b, deltaT)` which 
                                                - calculates `module_temperature = temperature.sapm_module(poa_global, temp_air, wind_speed, a, b)`
                                                - returns a call to `temperature.sapm_cell_from_module(module_temperature, poa_global, deltaT, irrad_ref)` which calculates and returns the cell temperture


                        - returns self

Thus for our manual determination of cell temp we will simply call temperature.sapm_cell with our own values.

In [39]:
poa_global = mc.results.total_irrad.poa_global
temp_air = mc.results.weather['temp_air']
wind_speed = mc.results.weather['wind_speed']
a = temperature_model_parameters['a']
b = temperature_model_parameters['b']
deltaT = temperature_model_parameters['deltaT']

cell_temp = pvlib.temperature.sapm_cell(poa_global, temp_air, wind_speed, a, b, deltaT)
print('cell_temp=', cell_temp)
print('mc.results.cell_temperature=', mc.results.cell_temperature)

cell_temp= 2022-04-08 12:10:00-07:00    66.780149
dtype: float64
mc.results.cell_temperature= 2022-04-08 12:10:00-07:00    66.780149
dtype: float64


## Manual V_oc and V_mpp Calculation

`model_chain.run_model(weather)` calls
- `self.effective_irradiance_model()` which
- `self.aoi_model()` which
- `self.spectral_model()`
- `self._run_from_effective_irrad(weather)` which binds `weather` to `data` ___ and calls
    - `self._prepare_temperature(data)` which ultimately inserts cell temps in `self.results.cell_temperature`
    - `self.dc_model()` which returns `self._dc_model` which was set during init to `self.desoto` which returns
        - `self._singlediode(self.system.calcparams_desoto)` which
            - binds to `calcparams_model_function` `self.system.calcparams_desoto`
            - binds `params` to the value returned by `calcparams_model_function(self.results.effective_irradiance, self.results.cell_temperature, unwrap=False)` which effectively calls `self.system.calcparams_desoto(self.results.effective_irradiance, self.results.cell_temperature, unwrap=False)` which is really calling
                - `PVSystem.calcparams_desoto(self.results.effective_irradiance, self.results.cell_temperature, unwrap=False)` which builds kwargs and calls
                    - `pvsystem.calcparams_desoto(effective_irradiance, temp_cell, alpha_sc, a_ref, I_L_ref, I_o_ref, R_sh_ref, R_s, EgRef=1.121, dEgdT=-0.0002677, irrad_ref=1000, temp_ref=25)` which returns a tuple of `photocurrent`, `saturation_current`, `resistance_series`, `resistance_shunt`, `nNsVth`
            - sets self.results.diode_params to a dataframe made from params of the form `{'I_L': photocurrent, 'I_o': saturation_current, 'R_s': resistance_series, 'R_sh': resistance_shunt,'nNsVth': nNsVth}`
            - sets self.results.dc to the value returned by `self.system.singlediode(params)` which calls `PVSystem.singlediode(self, photocurrent, saturation_current,resistance_series, resistance_shunt, nNsVth, ivcurve_pnts=None)`
                - which returns `pvsystem.singlediode(photocurrent, saturation_current,resistance_series, resistance_shunt, nNsVth, ivcurve_pnts=None, method='lambertw')` which for our case
                    - binds `args` to `(photocurrent, saturation_current, resistance_series,resistance_shunt, nNsVth)`
                    - binds `out` to `_singlediode._lambertw(args, ivcurve_pnts)` which is really `pvlib.singlediode_lambertw(photocurrent, saturation_current, resistance_series, resistance_shunt, nNsVth, ivcurve_pnts=None)` which
                        - calculates and returns a tuple `(i_sc, v_oc, i_mp, v_mp, p_mp, i_x, i_xx)`
                    - returns out as a dataframe (or dict of scalars) with columns `i_sc`, `v_oc`, `i_mp`, `v_mp`, `p_mp`, `i_x`, `i_xx`

We first need: 
`effective_irradiance`
`cell_temp`
`module_parameters`

Then we run:
`params = pvsystem.calcparams_desoto(effective_irradiance, cell_temp, ...module_parameters)`  

`params` is a tuple (`photocurrent`, `saturation_current`, `resistance_series`, `resistance_shunt`, `nNsVth`)

Then we run:
`something = pvlib.singlediode._lambertw(photocurrent, saturation_current, resistance_series, resistance_shunt, nNsVth)`

In [41]:
effective_irradiance = mc.results.effective_irradiance
print('effective_irradiance=', effective_irradiance)
cell_temp = mc.results.cell_temperature
print('cell_temp=', cell_temp)

# Desoto model parameters
alpha_sc = rich_solar_12V_params['alpha_sc']
a_ref = rich_solar_12V_params['a_ref']
I_L_ref = rich_solar_12V_params['I_L_ref']
I_o_ref = rich_solar_12V_params['I_o_ref']
R_sh_ref = rich_solar_12V_params['R_sh_ref']
R_s = rich_solar_12V_params['R_s']

print('alpha_sc=', alpha_sc)
print('a_ref=', a_ref)
print('I_L_ref=', I_L_ref)
print('I_o_ref=', I_o_ref)
print('R_sh_ref=', R_sh_ref)

photocurrent, saturation_current, resistance_series, resistance_shunt, nNsVth = pvlib.pvsystem.calcparams_desoto(effective_irradiance, cell_temp, alpha_sc, a_ref, I_L_ref, I_o_ref, R_sh_ref, R_s)
# get the values from the series
photocurrent = photocurrent.values[0]
saturation_current = saturation_current.values[0]
resistance_series = resistance_series.values[0]
resistance_shunt = resistance_shunt.values[0]
nNsVth = nNsVth.values[0]

print('photocurrent=', photocurrent)
print('saturation_current=', saturation_current)
print('resistance_series=', resistance_series)
print('resistance_shunt=', resistance_shunt)
print('nNsVth=', nNsVth)

# Calculate the diode parameters
something = pvlib.singlediode._lambertw(photocurrent, saturation_current, resistance_series, resistance_shunt, nNsVth)
someother = pvlib.pvsystem.singlediode(photocurrent, saturation_current, resistance_series, resistance_shunt, nNsVth, method='lambertw')

print('something=', something)
print('someother=', someother)

for key in someother:
    print(key, someother[key])

effective_irradiance= 2022-04-08 12:10:00-07:00    1014.276987
dtype: float64
cell_temp= 2022-04-08 12:10:00-07:00    66.780149
dtype: float64
alpha_sc= 0.006962
a_ref= 0.973804158972974
I_L_ref= 11.44031204496401
I_o_ref= 7.608255760344541e-10
R_sh_ref= 197.25444549468273
photocurrent= 11.898671422874457
saturation_current= 3.689758348690943e-07
resistance_series= 0.12270248066631148
resistance_shunt= 194.4778872299518
nNsVth= 1.1102646073366391
something= (11.891167890628282, 19.1860782074873, 10.930028986844647, 14.96877396817531, 163.6091333696817, 11.834161224231412, 7.679089521584184)
someother= {'i_sc': 11.891167890628282, 'v_oc': 19.1860782074873, 'i_mp': 10.930028986844647, 'v_mp': 14.96877396817531, 'p_mp': 163.6091333696817, 'i_x': 11.834161224231412, 'i_xx': 7.679089521584184}
i_sc 11.891167890628282
v_oc 19.1860782074873
i_mp 10.930028986844647
v_mp 14.96877396817531
p_mp 163.6091333696817
i_x 11.834161224231412
i_xx 7.679089521584184
